## Mace MD Demo notebook

This should demonstrate the bare bones of the API for setting up a variety of MD jobs with hybrid MACE-classical hamiltonians.  The same functionalty is exposed through the `mace-md` command line program that is installed with the `mace-openmm-interop` package


Your vs code server instance should have an attached CUDA GPU for this to run!

In [1]:
!nvidia-smi

Wed Jan  4 12:25:04 2023       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 510.85.02    Driver Version: 510.85.02    CUDA Version: 11.6     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  NVIDIA A100-SXM...  On   | 00000000:41:00.0 Off |                    0 |
| N/A   33C    P0    78W / 500W |      0MiB / 81920MiB |      0%      Default |
|                               |                      |             Disabled |
+-------------------------------+----------------------+----------------------+
                                                                               
+-------

In [1]:
from mace_openmm.mixed_system import MixedSystem
import torch
import logging
from typing import Optional, Union
from mace import tools

/rds/user/jhm72/hpc-work/openmmtools-dev/openmmtools/storage/iodrivers.py:88: SyntaxWarning: "is not" with a literal. Did you mean "!="?
  return '/'.join([path_part.strip('/ ') for path_part in split_path if path_part is not ''])
/rds/user/jhm72/hpc-work/openmmtools-dev/openmmtools/storage/iodrivers.py:703: SyntaxWarning: "is" with a literal. Did you mean "=="?
  if self._output_mode is 'a':


In [2]:
tools.setup_logger(level="DEBUG", directory="./logs")

In [3]:
# Change inputs in this cell only
file = "../tests/example_data/ejm_31.sdf"  # path to the system to be simulated, can be either protein ligand complex or isolated (sdf)
ml_mol = "../tests/example_data/ejm_31.sdf"  # path to the proper topology of the small molecule to be modeleld by mace, since pdb files are not sufficiently detailed.  this should be sdf.
model_path = "../tests/example_data/MACE_SPICE_larger.model"
forcefields = [
    "amber/protein.ff14SB.xml",
    "amber/tip3p_standard.xml",
    "amber14/DNA.OL15.xml",
]
# name of the residue to be parametrised by mace, should correspond to whatever it is called in the ml_mol sdf file and system file
resname = "4GIH_lig_ejm_31"
# minimum separation between the solute and box wall
padding = 1.2
# ion concentration in the solute
ionicStrength = 0.15
# cutoff distance (nm) for the non-bonded terns of the classical forcefield
nonbondedCutoff = 1.0
# name of the neural network potential, should correpond to the implemented FFs
# in the openmm-ml package
potential = "mace"
# simulation temperature
temperature = 298.15
# precision to use for both MACE and openMM
dtype = torch.float64
# which version of the torch_nl to use - the n^2 impl is more reliable
neighbour_list = "torch_nl_n2"
# directory where simulation output will be written
output_dir = "./output"
# specify the type of system to create - pure (just the solute simulated in vcuum)
# hybrid (small molecule modelled by MACE, rest of the system (protein or solvent) modelled by classical forcefield), decoupled (for ABFE simulations - lambda parameter controls switching off the ligand non-bonded terms)
system_type = "hybrid"

In [4]:
torch.set_default_dtype(dtype)

In [5]:
mixedSystem = MixedSystem(
    file=file,
    ml_mol=ml_mol,
    model_path=model_path,
    forcefields=forcefields,
    resname=resname,
    padding=padding,
    ionicStrength=ionicStrength,
    nonbondedCutoff=nonbondedCutoff,
    potential=potential,
    temperature=temperature,
    dtype=dtype,
    neighbour_list=neighbour_list,
    output_dir=output_dir,
    system_type=system_type,
)

2023-01-04 15:05:25.520 DEBUG: OpenMM will use Double precision
2023-01-04 15:05:25.520 INFO: Using openff-1.0 unconstrained forcefield
Initialized topology with (32, 3) positions
2023-01-04 15:05:25.971 INFO: Requested to generate parameters for residue <Residue 0 (4GIH_lig_ejm_31) of chain 0>
2023-01-04 15:05:26.421 INFO: Generating a residue template for [H][c]1[n][c]([N]([H])[C](=[O])[C]([H])([H])[H])[c]([H])[c]([N]([H])[C](=[O])[c]2[c]([Cl])[c]([H])[c]([H])[c]([H])[c]2[Cl])[c]1[H] using openff_unconstrained-1.0.0.offxml
2023-01-04 15:05:27.792 DEBUG: Changing app registry from <pint.registry.LazyRegistry object at 0x7f444a3cde50> to <pint.registry.UnitRegistry object at 0x7f444837a9d0>.
2023-01-04 15:05:27.796 DEBUG: Generating parameters...
2023-01-04 15:05:27.805 DEBUG: Finding matches for ProperTorsionHandler
2023-01-04 15:05:27.806 DEBUG: [*:1]-[#6X4:2]-[#6X4:3]-[*:4]                                    :        0 matches
2023-01-04 15:05:27.808 DEBUG: [#6X4:1]-[#6X4:2]-[#6X4:3

2023-01-04 15:05:27.898 DEBUG: [#6X3:1]=[#7X2,#7X3+1:2]-[#6X4:3]-[#1:4]                         :        0 matches
2023-01-04 15:05:27.899 DEBUG: [#6X3:1]=[#7X2,#7X3+1:2]-[#6X4:3]-[#6X3,#6X4:4]                  :        0 matches
2023-01-04 15:05:27.901 DEBUG: [*:1]~[#7X3,#7X2-1:2]-[#6X3:3]~[*:4]                             :       16 matches
2023-01-04 15:05:27.903 DEBUG: [*:1]~[#7X3,#7X2-1:2]-!@[#6X3:3]~[*:4]                           :       16 matches
2023-01-04 15:05:27.904 DEBUG: [#1:1]-[#7X3:2]-[#6X3:3]=[#8,#16,#7:4]                           :        2 matches
2023-01-04 15:05:27.905 DEBUG: [*:1]-[#7X3;r5:2]-@[#6X3;r5:3]~[*:4]                             :        0 matches
2023-01-04 15:05:27.907 DEBUG: [#8X1:1]~[#7X3:2]~[#6X3:3]~[*:4]                                 :        0 matches
2023-01-04 15:05:27.908 DEBUG: [*:1]=[#7X2,#7X3+1:2]-[#6X3:3]-[*:4]                             :        0 matches
2023-01-04 15:05:27.909 DEBUG: [*:1]=[#7X2,#7X3+1:2]-[#6X3:3]=,:[*:4]           

2023-01-04 15:05:27.996 DEBUG: [#6X3:1]-[#16X4,#16X3+0:2]-[#7X4,#7X3:3]-[#6X4:4]                :        0 matches
2023-01-04 15:05:27.997 DEBUG: [#8X1:1]~[#16X4,#16X3+0:2]-[#7X4,#7X3:3]-[#1:4]                  :        0 matches
2023-01-04 15:05:27.999 DEBUG: [#8X1:1]~[#16X4,#16X3+0:2]-[#7X4,#7X3:3]-[#6X4:4]                :        0 matches
2023-01-04 15:05:28.000 DEBUG: [#6X3:1]-[#16X4,#16X3+0:2]-[#7X3:3]-[#6X3:4]                     :        0 matches
2023-01-04 15:05:28.001 DEBUG: [#6X4:1]-[#16X4,#16X3+0:2]-[#7X3:3]-[#6X3:4]                     :        0 matches
2023-01-04 15:05:28.003 DEBUG: [#8X1:1]~[#16X4,#16X3+0:2]-[#7X3:3]-[#6X3:4]                     :        0 matches
2023-01-04 15:05:28.004 DEBUG: [#8X1:1]~[#16X4,#16X3+0:2]-[#7X3:3]-[#7X2:4]                     :        0 matches
2023-01-04 15:05:28.005 DEBUG: [*:1]~[#16X4,#16X3+0:2]=,:[#7X2:3]-,:[*:4]                       :        0 matches
2023-01-04 15:05:28.007 DEBUG: [#6X4:1]-[#16X4,#16X3+0:2]-[#7X2:3]~[#6X3:4]     

2023-01-04 15:05:28.094 DEBUG: [#6X3:1]-[#7X3:2]                                                :        4 matches
2023-01-04 15:05:28.095 DEBUG: [#6X4:1]-[#7X3:2]-[#6X3]=[#8X1+0]                                :        0 matches
2023-01-04 15:05:28.097 DEBUG: [#6X3:1](=[#8X1+0])-[#7X3:2]                                     :        2 matches
2023-01-04 15:05:28.098 DEBUG: [#6X3:1]-[#7X2:2]                                                :        0 matches
2023-01-04 15:05:28.100 DEBUG: [#6X3:1]:[#7X2,#7X3+1:2]                                         :        2 matches
2023-01-04 15:05:28.101 DEBUG: [#6X3:1]=[#7X2,#7X3+1:2]                                         :        0 matches
2023-01-04 15:05:28.102 DEBUG: [#6:1]-[#8:2]                                                    :        0 matches
2023-01-04 15:05:28.104 DEBUG: [#6X4:1]-[#8X2H0:2]                                              :        0 matches
2023-01-04 15:05:28.105 DEBUG: [#6X3:1]-[#8X2:2]                                

2023-01-04 15:05:28.190 DEBUG: [#15:1]-[#35:2]                                                  :        0 matches
2023-01-04 15:05:28.191 DEBUG: [#15:1]-[#53:2]                                                  :        0 matches
2023-01-04 15:05:28.192 DEBUG: [#6X4:1]-[#1:2]                                                  :        3 matches
2023-01-04 15:05:28.194 DEBUG: [#6X3:1]-[#1:2]                                                  :        6 matches
2023-01-04 15:05:28.195 DEBUG: [#6X2:1]-[#1:2]                                                  :        0 matches
2023-01-04 15:05:28.196 DEBUG: [#7:1]-[#1:2]                                                    :        2 matches
2023-01-04 15:05:28.198 DEBUG: [#8:1]-[#1:2]                                                    :        0 matches
2023-01-04 15:05:28.198 DEBUG: 33 matches identified
2023-01-04 15:05:28.199 INFO: 33 bonds added (0 skipped due to constraints)
2023-01-04 15:05:28.200 DEBUG: Finding matches for AngleHandler
20

In [6]:
# Once the mixed system is created, we can run several different types of simulation: 

mixedSystem.run_mixed_md(steps=1000, interval=100, output_file="./output_md_test.pdb")

2023-01-04 15:06:17.915 INFO: Minimising energy...


[W parser.cpp:3513] Warning: operator() sees varying value in profiling, ignoring and this should be handled by GUARD logic (function operator())
[W graph_fuser.cpp:104] Warning: operator() profile_node %512 : int[] = prim::profile_ivalue(%510)
 does not have profile information (function operator())


#"Step","Time (ps)","Potential Energy (kJ/mole)","Temperature (K)","Speed (ns/day)"
100,0.10000000000000007,-23648.32740856873,30.416865584839044,0
200,0.20000000000000015,-23336.1411348361,57.08408744104098,4.7
300,0.3000000000000002,-23079.20481705203,77.5377090375836,4.69
400,0.4000000000000003,-22797.46837060529,97.1923950995527,4.7
500,0.5000000000000003,-22550.34228893614,118.69151187954135,4.68
600,0.6000000000000004,-22392.828543652693,137.0635894462519,4.7
700,0.7000000000000005,-22134.124148869043,145.9986779638119,4.7
800,0.8000000000000006,-22004.613216828613,166.46023021788596,4.7
900,0.9000000000000007,-21753.877434927577,171.91864781828116,4.7
1000,1.0000000000000007,-21698.149047767365,185.1024845506458,4.7


In [ ]:
# runs Markov chain monte carlo replica exchange to interpoolate between MM and MM/ML descriptions of the system
mixedSystem.run_replex_equilibrium_fep(replicas=5, restart=False, steps=10)

2023-01-04 15:10:22.881 INFO: Starting Repex sampling from scratch
2023-01-04 15:10:22.882 WARNING: Warning: The openmmtools.multistate API is experimental and may change in future releases
2023-01-04 15:10:22.921 DEBUG: CUDA devices available: (['0', ' NVIDIA A100-SXM4-80GB'],)
2023-01-04 15:10:53.732 INFO: making lambda states...
running thermolist population...
running lambda subinterval 0.0.
this subinterval matched; adding to state...
running lambda subinterval 0.1111111111111111.
running lambda subinterval 0.2222222222222222.
running lambda subinterval 0.3333333333333333.
running lambda subinterval 0.4444444444444444.
running lambda subinterval 0.5555555555555556.
running lambda subinterval 0.6666666666666666.
running lambda subinterval 0.7777777777777777.
running lambda subinterval 0.8888888888888888.
running lambda subinterval 1.0.
this subinterval matched; adding to state...
2023-01-04 15:11:12.220 WARNING: Warning: The openmmtools.multistate API is experimental and may change

2023-01-04 15:12:46.610 DEBUG: pending: deque([State(pc_initial=0 nstack_initial=0)])
2023-01-04 15:12:46.611 DEBUG: stack: []
2023-01-04 15:12:46.611 DEBUG: dispatch pc=0, inst=NOP(arg=None, lineno=321)
2023-01-04 15:12:46.612 DEBUG: stack []
2023-01-04 15:12:46.612 DEBUG: dispatch pc=2, inst=LOAD_GLOBAL(arg=0, lineno=321)
2023-01-04 15:12:46.612 DEBUG: stack []
2023-01-04 15:12:46.613 DEBUG: dispatch pc=4, inst=LOAD_FAST(arg=0, lineno=321)
2023-01-04 15:12:46.613 DEBUG: stack ['$2load_global.0']
2023-01-04 15:12:46.613 DEBUG: dispatch pc=6, inst=CALL_FUNCTION(arg=1, lineno=321)
2023-01-04 15:12:46.613 DEBUG: stack ['$2load_global.0', '$nswap_attempts4.1']
2023-01-04 15:12:46.614 DEBUG: dispatch pc=8, inst=GET_ITER(arg=None, lineno=321)
2023-01-04 15:12:46.614 DEBUG: stack ['$6call_function.2']
2023-01-04 15:12:46.614 DEBUG: end state. edges=[Edge(pc=10, stack=('$8get_iter.3',), blockstack=(), npush=0)]
2023-01-04 15:12:46.615 DEBUG: pending: deque([State(pc_initial=10 nstack_initial=

2023-01-04 15:12:46.639 DEBUG: dispatch pc=88, inst=STORE_FAST(arg=13, lineno=334)
2023-01-04 15:12:46.639 DEBUG: stack ['$phi12.0', '$86binary_subscr.32']
2023-01-04 15:12:46.640 DEBUG: dispatch pc=90, inst=LOAD_FAST(arg=3, lineno=335)
2023-01-04 15:12:46.640 DEBUG: stack ['$phi12.0']
2023-01-04 15:12:46.640 DEBUG: dispatch pc=92, inst=LOAD_FAST(arg=8, lineno=335)
2023-01-04 15:12:46.641 DEBUG: stack ['$phi12.0', '$_energy_thermodynamic_states90.33']
2023-01-04 15:12:46.641 DEBUG: dispatch pc=94, inst=LOAD_FAST(arg=10, lineno=335)
2023-01-04 15:12:46.641 DEBUG: stack ['$phi12.0', '$_energy_thermodynamic_states90.33', '$replica_j92.34']
2023-01-04 15:12:46.641 DEBUG: dispatch pc=96, inst=BUILD_TUPLE(arg=2, lineno=335)
2023-01-04 15:12:46.642 DEBUG: stack ['$phi12.0', '$_energy_thermodynamic_states90.33', '$replica_j92.34', '$thermodynamic_state_j94.35']
2023-01-04 15:12:46.642 DEBUG: dispatch pc=98, inst=BINARY_SUBSCR(arg=None, lineno=335)
2023-01-04 15:12:46.642 DEBUG: stack ['$phi12.

2023-01-04 15:12:46.664 DEBUG: dispatch pc=176, inst=LOAD_GLOBAL(arg=1, lineno=343)
2023-01-04 15:12:46.664 DEBUG: stack ['$phi168.0', '$174call_method.4']
2023-01-04 15:12:46.664 DEBUG: dispatch pc=178, inst=LOAD_METHOD(arg=5, lineno=343)
2023-01-04 15:12:46.664 DEBUG: stack ['$phi168.0', '$174call_method.4', '$176load_global.5']
2023-01-04 15:12:46.665 DEBUG: dispatch pc=180, inst=LOAD_FAST(arg=15, lineno=343)
2023-01-04 15:12:46.665 DEBUG: stack ['$phi168.0', '$174call_method.4', '$178load_method.6']
2023-01-04 15:12:46.665 DEBUG: dispatch pc=182, inst=CALL_METHOD(arg=1, lineno=343)
2023-01-04 15:12:46.665 DEBUG: stack ['$phi168.0', '$174call_method.4', '$178load_method.6', '$log_p_accept180.7']
2023-01-04 15:12:46.666 DEBUG: dispatch pc=184, inst=COMPARE_OP(arg=0, lineno=343)
2023-01-04 15:12:46.666 DEBUG: stack ['$phi168.0', '$174call_method.4', '$182call_method.8']
2023-01-04 15:12:46.666 DEBUG: dispatch pc=186, inst=POP_JUMP_IF_FALSE(arg=10, lineno=343)
2023-01-04 15:12:46.667 D

2023-01-04 15:12:46.695 DEBUG: changing phismap: defaultdict(<class 'set'>,
            {'$phi10.0': {('$8get_iter.3',
                           State(pc_initial=0 nstack_initial=0)),
                          ('$phi12.0', State(pc_initial=12 nstack_initial=2))},
             '$phi12.0': {('$8get_iter.3',
                           State(pc_initial=0 nstack_initial=0)),
                          ('$phi12.0', State(pc_initial=12 nstack_initial=2))},
             '$phi12.1': {('$10for_iter.2',
                           State(pc_initial=10 nstack_initial=1))},
             '$phi168.0': {('$8get_iter.3',
                            State(pc_initial=0 nstack_initial=0))},
             '$phi188.0': {('$8get_iter.3',
                            State(pc_initial=0 nstack_initial=0))}})
2023-01-04 15:12:46.696 DEBUG: changing phismap: defaultdict(<class 'set'>,
            {'$phi10.0': {('$8get_iter.3',
                           State(pc_initial=0 nstack_initial=0))},
             '$phi12.0'

2023-01-04 15:12:46.699 DEBUG: block_infos State(pc_initial=168 nstack_initial=1):
AdaptBlockInfo(insts=((168, {'res': '$168load_global.1'}), (170, {'item': '$168load_global.1', 'res': '$170load_attr.2'}), (172, {'item': '$170load_attr.2', 'res': '$172load_method.3'}), (174, {'func': '$172load_method.3', 'args': [], 'res': '$174call_method.4'}), (176, {'res': '$176load_global.5'}), (178, {'item': '$176load_global.5', 'res': '$178load_method.6'}), (180, {'res': '$log_p_accept180.7'}), (182, {'func': '$178load_method.6', 'args': ['$log_p_accept180.7'], 'res': '$182call_method.8'}), (184, {'lhs': '$174call_method.4', 'rhs': '$182call_method.8', 'res': '$184compare_op.9'}), (186, {'pred': '$184compare_op.9'})), outgoing_phis={}, blockstack=(), active_try_block=None, outgoing_edgepushed={188: ('$phi168.0',), 10: ('$phi168.0',)})
2023-01-04 15:12:46.699 DEBUG: block_infos State(pc_initial=188 nstack_initial=1):
AdaptBlockInfo(insts=((188, {'res': '$thermodynamic_state_j188.1'}), (190, {'res'

2023-01-04 15:12:46.722 DEBUG: ==== SSA block analysis pass on 0
2023-01-04 15:12:46.722 DEBUG: Running <numba.core.ssa._GatherDefsHandler object at 0x7f425999ae50>
2023-01-04 15:12:46.723 DEBUG: on stmt: nswap_attempts = arg(0, name=nswap_attempts)
2023-01-04 15:12:46.723 DEBUG: on stmt: n_replicas = arg(1, name=n_replicas)
2023-01-04 15:12:46.724 DEBUG: on stmt: _replica_thermodynamic_states = arg(2, name=_replica_thermodynamic_states)
2023-01-04 15:12:46.724 DEBUG: on stmt: _energy_thermodynamic_states = arg(3, name=_energy_thermodynamic_states)
2023-01-04 15:12:46.724 DEBUG: on stmt: _n_accepted_matrix = arg(4, name=_n_accepted_matrix)
2023-01-04 15:12:46.725 DEBUG: on stmt: _n_proposed_matrix = arg(5, name=_n_proposed_matrix)
2023-01-04 15:12:46.725 DEBUG: on stmt: $2load_global.0 = global(range: <class 'range'>)
2023-01-04 15:12:46.725 DEBUG: on stmt: $6call_function.2 = call $2load_global.0(nswap_attempts, func=$2load_global.0, args=[Var(nswap_attempts, replicaexchange.py:321)],

2023-01-04 15:12:46.743 DEBUG: on stmt: $184compare_op.9 = $174call_method.4 < $182call_method.8
2023-01-04 15:12:46.743 DEBUG: on stmt: bool186 = global(bool: <class 'bool'>)
2023-01-04 15:12:46.743 DEBUG: on stmt: $186pred = call bool186($184compare_op.9, func=bool186, args=(Var($184compare_op.9, replicaexchange.py:343),), kws=(), vararg=None, varkwarg=None, target=None)
2023-01-04 15:12:46.744 DEBUG: on stmt: branch $186pred, 188, 247
2023-01-04 15:12:46.744 DEBUG: ==== SSA block analysis pass on 188
2023-01-04 15:12:46.744 DEBUG: Running <numba.core.ssa._GatherDefsHandler object at 0x7f425999ae50>
2023-01-04 15:12:46.744 DEBUG: on stmt: _replica_thermodynamic_states[replica_i] = thermodynamic_state_j
2023-01-04 15:12:46.745 DEBUG: on stmt: _replica_thermodynamic_states[replica_j] = thermodynamic_state_i
2023-01-04 15:12:46.745 DEBUG: on stmt: $210build_tuple.10 = build_tuple(items=[Var(thermodynamic_state_i, replicaexchange.py:328), Var(thermodynamic_state_j, replicaexchange.py:329

2023-01-04 15:12:46.751 DEBUG: SSA violators set()
2023-01-04 15:12:46.762 DEBUG: bytecode dump:
>          0	NOP(arg=None, lineno=3)
           2	LOAD_GLOBAL(arg=0, lineno=3)
           4	LOAD_FAST(arg=0, lineno=3)
           6	CALL_FUNCTION(arg=1, lineno=3)
           8	RETURN_VALUE(arg=None, lineno=3)
2023-01-04 15:12:46.763 DEBUG: pending: deque([State(pc_initial=0 nstack_initial=0)])
2023-01-04 15:12:46.763 DEBUG: stack: []
2023-01-04 15:12:46.763 DEBUG: dispatch pc=0, inst=NOP(arg=None, lineno=3)
2023-01-04 15:12:46.763 DEBUG: stack []
2023-01-04 15:12:46.764 DEBUG: dispatch pc=2, inst=LOAD_GLOBAL(arg=0, lineno=3)
2023-01-04 15:12:46.764 DEBUG: stack []
2023-01-04 15:12:46.764 DEBUG: dispatch pc=4, inst=LOAD_FAST(arg=0, lineno=3)
2023-01-04 15:12:46.765 DEBUG: stack ['$2load_global.0']
2023-01-04 15:12:46.765 DEBUG: dispatch pc=6, inst=CALL_FUNCTION(arg=1, lineno=3)
2023-01-04 15:12:46.765 DEBUG: stack ['$2load_global.0', '$low4.1']
2023-01-04 15:12:46.765 DEBUG: dispatch pc=8, i

2023-01-04 15:12:46.848 DEBUG: SSA violators set()
2023-01-04 15:12:46.850 DEBUG: bytecode dump:
>          0	NOP(arg=None, lineno=3)
           2	LOAD_GLOBAL(arg=0, lineno=3)
           4	CALL_FUNCTION(arg=0, lineno=3)
           6	RETURN_VALUE(arg=None, lineno=3)
2023-01-04 15:12:46.851 DEBUG: pending: deque([State(pc_initial=0 nstack_initial=0)])
2023-01-04 15:12:46.851 DEBUG: stack: []
2023-01-04 15:12:46.851 DEBUG: dispatch pc=0, inst=NOP(arg=None, lineno=3)
2023-01-04 15:12:46.852 DEBUG: stack []
2023-01-04 15:12:46.852 DEBUG: dispatch pc=2, inst=LOAD_GLOBAL(arg=0, lineno=3)
2023-01-04 15:12:46.852 DEBUG: stack []
2023-01-04 15:12:46.852 DEBUG: dispatch pc=4, inst=CALL_FUNCTION(arg=0, lineno=3)
2023-01-04 15:12:46.853 DEBUG: stack ['$2load_global.0']
2023-01-04 15:12:46.853 DEBUG: dispatch pc=6, inst=RETURN_VALUE(arg=None, lineno=3)
2023-01-04 15:12:46.853 DEBUG: stack ['$4call_function.1']
2023-01-04 15:12:46.854 DEBUG: end state. edges=[]
2023-01-04 15:12:46.854 DEBUG: --------